# Corynocarpus cribbianus
## ONT Basecalling

### Run statistics (Before Basecalling):
- LSK114 kit
- N50 = 18kb
- Reads = 1.15M
- Bases = 10.68 Gbases
- Data = 116.27 GB
- Coverage = ~15X

In [8]:
WKDIR=/workspace/hraijc/Karaka/Ccribbianus/01_basecalling
READDIR=/input/genomic/plant/Corynocarpus/cribbianus/Ccrib_ONT1/20240919_1351_MN19482_FAW89114_3a35e5ac/pod5/
DORADO=/workspace/hraijc/git_clones/dorado-0.7.3-linux-x64/bin/dorado
PREFIX=Ccrib_ONT1

In [9]:
cd $WKDIR
mkdir -p ${WKDIR}/log

In [4]:
$DORADO -v

[2024-09-24 10:58:40.240] [info] Running: "-v"
0.7.3+6e6c45c


In [5]:
$DORADO basecaller -h

[2024-09-24 10:58:53.234] [info] Running: "basecaller" "-h"
Usage: dorado [-h] [--device VAR] [--read-ids VAR] [--resume-from VAR] [--max-reads VAR] [--min-qscore VAR] [--batchsize VAR] [--chunksize VAR] [--overlap VAR] [--recursive] [--modified-bases VAR...] [--modified-bases-models VAR] [--modified-bases-threshold VAR] [--emit-fastq] [--emit-sam] [--emit-moves] [--reference VAR] [--kit-name VAR] [--barcode-both-ends] [--no-trim] [--trim VAR] [--sample-sheet VAR] [--barcode-arrangement VAR] [--barcode-sequences VAR] [--primer-sequences VAR] [--estimate-poly-a] [--poly-a-config VAR] [-k VAR] [-w VAR] [-I VAR] [--secondary VAR] [-N VAR] [-Y] [--bandwidth VAR] [--junc-bed VAR] [--mm2-preset VAR] model data

Positional arguments:
  model                     	model selection {fast,hac,sup}@v{version} for automatic model selection including modbases, or path to existing model directory 
  data                      	the data directory or file (POD5/FAST5 format). 

Optional arguments:
  -h, 

In [7]:
#DORADO duplex
sbatch << EOF
#!/bin/bash -e
#SBATCH -J basecalling
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=160G
#SBATCH --output=${WKDIR}/log/%J.out
#SBATCH --error=${WKDIR}/log/%J.err
#SBATCH --time=47:00:00
#SBATCH --gres=gpu:1

cd ${WKDIR}

${DORADO} duplex sup ${READDIR} > ${PREFIX}_duplex.bam


EOF

sbatch: slurm_job_submit: Job assigned to GPU nodes: gpu
Submitted batch job 7711926


In [10]:
du -sh ${WKDIR}/*.bam


11G	/workspace/hraijc/Karaka/Ccribbianus/01_basecalling/Ccrib_ONT1_duplex.bam


In [11]:
#DoradoDuplex ATD
seff 7711926

Job ID: 7711926
Cluster: powerplant
User/Group: hraijc/hraijc
State: COMPLETED (exit code 0)
Nodes: 1
Cores per node: 8
CPU Utilized: 08:28:06
CPU Efficiency: 15.88% of 2-05:19:36 core-walltime
Job Wall-clock time: 06:39:57
Memory Utilized: 147.24 GB
Memory Efficiency: 92.02% of 160.00 GB


In [12]:
# Get QC report of runs. 

sbatch << EOF
#!/bin/bash
#SBATCH -J NanoPlot
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=2
#SBATCH --mem-per-cpu=2G
#SBATCH --time=01:02:00




~/.local/bin/NanoPlot -t 2 --ubam ${PREFIX}_duplex.bam -o ${WKDIR}/${PREFIX}_summary-plots


EOF

Submitted batch job 7712379


In [13]:
# Bam -> fastq samtools
# Filtering using seqkit
# Stats


sbatch << EOF
#!/bin/bash
#SBATCH -J fastq_filter
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=12
#SBATCH --mem-per-cpu=8G
#SBATCH --time=04:02:00

module load samtools/1.20
cd ${WKDIR}

samtools fastq -@ 12 -T '*' ${PREFIX}_duplex.bam > ${PREFIX}_all.fastq
/workspace/hraijc/git_clones/seqkit/seqkit seq ${PREFIX}_all.fastq -j 12 -Q 15 -m 1000 > ${PREFIX}_q15L1k.fastq
/workspace/hraijc/git_clones/seqkit/seqkit stats *.fastq

EOF

Submitted batch job 7712383


In [15]:
cat ${WKDIR}/log/7712383.out

file                     format  type   num_seqs         sum_len  min_len  avg_len  max_len
Ccrib_ONT1_all.fastq     FASTQ   DNA   1,225,607  11,499,466,526        5  9,382.7  884,071
Ccrib_ONT1_q15L1k.fastq  FASTQ   DNA     577,027   6,543,494,551    1,000   11,340  304,232


In [16]:
ls $WKDIR

Ccrib_ONT1_all.fastq   Ccrib_ONT1_duplex.bam.bai  Ccrib_ONT1_summary-plots
Ccrib_ONT1_duplex.bam  Ccrib_ONT1_q15L1k.fastq	  log


In [17]:
# GZIP


sbatch << EOF
#!/bin/bash
#SBATCH -J gzip
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=8G
#SBATCH --time=01:02:00

cd ${WKDIR}

gzip *.fastq

EOF

Submitted batch job 7712612
